In [39]:
import numpy as np
from tnestmodel.datasets import datasets
from tnestmodel.temp_wl import compute_d_rounds
from tnestmodel.temp_fast_graph import SparseTempFastGraph
from nestmodel.graph_properties import number_of_flips_possible
import matplotlib.pyplot as plt

In [40]:
use_range_times = False

In [41]:
def switch_in_out(E):
    out = np.empty_like(E)
    out[:,0] = E[:,1]
    out[:,1] = E[:,0]
    out[:,2] = E[:,2]
    return out

In [42]:
#%load_ext snakeviz

In [43]:
#%%snakeviz --new-tab
Gs = []
time_deltas = []
hs = []
for dataset in datasets:
    print()
    print(dataset.name)
    E = dataset.read_edges()
    
    if np.any(E[:,0]==E[:,1]):
        print("self")
    if dataset.is_directed:
        E = switch_in_out(E)

    nodes = np.unique(E[:, :2].ravel())
    total_time_diff = np.max(E[:,2])-np.min(E[:,2])
    time_deltas.append(total_time_diff)
    h2 = int(round(total_time_diff*1.2))

    G = SparseTempFastGraph.from_temporal_edges(E, dataset.is_directed)
    if use_range_times:
        G.times = np.arange(len(G.times))
        h2 = int(round(len(G.times*1.2)))
    G.assign_colors_to_slices(h=h2)
        
    hs.append(h2)
    Gs.append(G)
    #break



aves-weaver-social
max_d 5

ht09
max_d 5

workplace_2013
max_d 5

mammalia-primate-association
max_d 3

mammalia-raccoon-proximity
max_d 4

talk_eo
max_d 4

dnc
max_d 5

fb-forum
max_d 5

email-eu2
max_d 5

email-eu3
max_d 4


In [44]:
#for G in Gs:
#    for d_iter in range(len(G.slices[0].base_partitions)):
#        G.s.reset_colors(d_iter, G.h, mode="global")
#        print(d_iter)
#        for t, G_t in zip(G.times,G.slices):
#            G.s.advance_time(t)
#            #print(G.s.current_colors[G_t.global_nodes], G_t.base_partitions[d_iter])
#            assert_array_equal(G.s.current_colors[G_t.global_nodes], G_t.base_partitions[d_iter])


In [45]:
from tnestmodel.temp_properties import NumberOfTrianglesCalculator, EdgePersistenceCalculator

In [46]:
G = SparseTempFastGraph.from_temporal_edges(np.array([(0,1,0), (1,2,1), (1,3,1), (2,0,2), (3,0,3), (2,0,4)],dtype=int), is_directed=True)
calculator = NumberOfTrianglesCalculator(G)
calculator.prepare()
# print(calculator.future_nodes_count, calculator.future_nodes)
G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True, dtype=int)

(array([0, 1, 2, 3, 4]), array([0, 3, 0, 0, 0]))

In [47]:
G = SparseTempFastGraph.from_temporal_edges(np.array([(0,1,0), (0,1,1), (1,2,2), (1,3,2), (2,0,3), (3,0,4), (2,0,5)],dtype=int), is_directed=True)
calculator = NumberOfTrianglesCalculator(G)
calculator.prepare()
# print(calculator.future_nodes_count, calculator.future_nodes)
G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True, dtype=int)

(array([0, 1, 2, 3, 4, 5]), array([0, 0, 6, 0, 0, 0]))

In [48]:
G = SparseTempFastGraph.from_temporal_edges(np.array([(0,1,0), (1,2,0), (2,0,0)],dtype=int), is_directed=True)
calculator = NumberOfTrianglesCalculator(G, strict=False)
calculator.prepare()
# print(calculator.future_nodes_count, calculator.future_nodes)
G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True, dtype=int)

(array([0]), array([3]))

In [49]:
ts = []
for h, G in zip(hs, Gs):
    t = G.times
    t = np.array(t, dtype=np.float64)
    t = t-t.min()
    t /= t.max()
    ts.append(t)

In [50]:
arrs = []
for h, G in zip(hs, Gs):
    calculator = NumberOfTrianglesCalculator(G, strict=True)
    calculator.prepare()
    _, arr = G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True, dtype=int)
    arrs.append(arr)

In [51]:
ts = []
pers = []
for h, G in zip(hs, Gs):
    calculator = EdgePersistenceCalculator(G)
    calculator.prepare()
    _, arr = G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True)
    print(arr.sum())
    pers.append(arr.sum())

0.0
18540.51996215101
11556.261380665277
112.68775679237598
471.7630397505978
5561.363961030679
716.9380535381002
1322.0
2807.110314831705
725.0545856087615


In [52]:
from tnestmodel.temp_properties import _number_of_edges

In [53]:


list_num_edges = []
for G in Gs:
    t, arr = G.compute_for_each_slice(_number_of_edges, min_size=1, call_with_time=False)
    list_num_edges.append(arr.sum())

In [54]:
rows = []
for arr, num_edges, dataset in zip(arrs, list_num_edges, datasets):
    
    print(dataset.name, "\t\tnum_triangles_per_edge", np.sum(arr)/num_edges)
    rows.append((dataset.name, arr.sum(), num_edges, np.sum(arr)/num_edges))

aves-weaver-social 		num_triangles_per_edge 0.0
ht09 		num_triangles_per_edge 2084.1175425112883
workplace_2013 		num_triangles_per_edge 1274.7187340999287
mammalia-primate-association 		num_triangles_per_edge 103.15
mammalia-raccoon-proximity 		num_triangles_per_edge 313.91837756634953
talk_eo 		num_triangles_per_edge 505.8582817609383
dnc 		num_triangles_per_edge 1082.689601916349
fb-forum 		num_triangles_per_edge 0.6467846327415236
email-eu2 		num_triangles_per_edge 16695.920604216473
email-eu3 		num_triangles_per_edge 302.69496307360384


In [55]:
from tnestmodel.temp_properties import get_edge_persistence, get_total_number_of_edges, numba_get_all_triangles, causal_triangles

In [56]:
import pandas as pd

In [57]:
pd.DataFrame.from_records(rows, columns=["name", "forward_triangles", "edges", "triangles per edge"])

,name,forward_triangles,edges,triangles per edge
0,aves-weaver-social,0,1333.0,0.000000
1,ht09,43387159,20818.0,2084.117543
2,workplace_2013,12526661,9827.0,1274.718734
3,mammalia-primate-association,138221,1340.0,103.150000
4,mammalia-raccoon-proximity,626895,1997.0,313.918378
5,talk_eo,18718274,37003.0,505.858282
6,dnc,34350493,31727.0,1082.689602
7,fb-forum,21785,33682.0,0.646785
8,email-eu2,768179307,46010.0,16695.920604
9,email-eu3,3647777,12051.0,302.694963


In [58]:
nodes_per = [G.num_nodes for G in Gs]
times_per = [len(G.slices) for G in Gs]
directed_per = [G.is_directed for G in Gs]

In [59]:
for dataset, G in zip(datasets, Gs):
    E = G.to_temporal_edges()
    E2 = set(map(tuple, E))
    print(E.shape[0], len(E2), get_total_number_of_edges(G))

1333 1333 1333
20818 20818 20818
9827 9827 9827
1340 1340 1340
1997 1997 1997
37003 37003 37003
31727 31727 31727
33682 33682 33682
46010 46010 46010
12051 12051 12051


In [60]:
counts = []
for dataset, G in zip(datasets, Gs):
    if dataset.name=="hospital_ward":
        G_out = G
    
    c = numba_get_all_triangles(G)
    
    
    print(dataset, c)
    counts.append(c)


CSVDataset(aves-weaver-social, False) 12366
CSVDataset(ht09, False) 559040718
CSVDataset(workplace_2013, False) 140416356
CSVDataset(mammalia-primate-association, False) 2229750
CSVDataset(mammalia-raccoon-proximity, False) 8621310
CSVDataset(talk_eo, True) 110333082
CSVDataset(dnc, True) 203675919
CSVDataset(fb-forum, True) 133227
CSVDataset(email-eu2, True) 4574681847
CSVDataset(email-eu3, True) 21498570


In [61]:
559040718/6

93173453.0

In [62]:
df = pd.DataFrame.from_records(rows, columns=["name", "forward_triangles", "edges", "triangles per edge"])
df["forward_triangles"] = np.array(df["forward_triangles"], dtype=np.float64)
df["dir"] = directed_per
df["pers"] = pers
counts = np.array(counts, dtype=np.float64)
inds = df["dir"]
counts[inds]/=3
counts[~inds]/=6
df["agg_triangles"]=counts
df["ratio"] = df["forward_triangles"]/df["agg_triangles"]




# divide by 3 in the directed case?
df["nodes"] = nodes_per
df["times"] = times_per

In [63]:
df["agg_triangles_edges"] = df["agg_triangles"]/(df["edges"])
df["agg_triangles_VT"] = df["agg_triangles"]/(df["nodes"]*df["times"])
df["pers_E"] = df["pers"]/(df["edges"])/2
df

,name,forward_triangles,edges,triangles per edge,dir,pers,agg_triangles,ratio,nodes,times,agg_triangles_edges,agg_triangles_VT,pers_E
0,aves-weaver-social,0.0,1333.0,0.000000,False,0.000000,2.061000e+03,0.000000,445,23,1.546137,0.201368,0.000000
1,ht09,43387159.0,20818.0,2084.117543,False,18540.519962,9.317345e+07,0.465660,113,5246,4475.619800,157.175721,0.445300
2,workplace_2013,12526661.0,9827.0,1274.718734,False,11556.261381,2.340273e+07,0.535265,92,7104,2381.472067,35.807637,0.587985
3,mammalia-primate-association,138221.0,1340.0,103.150000,False,112.687757,3.716250e+05,0.371937,25,19,277.332090,782.368421,0.042048
4,mammalia-raccoon-proximity,626895.0,1997.0,313.918378,False,471.763040,1.436885e+06,0.436288,24,52,719.521783,1151.350160,0.118118
5,talk_eo,18718274.0,37003.0,505.858282,True,5561.363961,3.677769e+07,0.508957,7585,32352,993.911142,0.149875,0.075147
6,dnc,34350493.0,31727.0,1082.689602,True,716.938054,6.789197e+07,0.505958,1891,18682,2139.880008,1.921779,0.011299
7,fb-forum,21785.0,33682.0,0.646785,True,1322.000000,4.440900e+04,0.490554,899,33483,1.318479,0.001475,0.019625
8,email-eu2,768179307.0,46010.0,16695.920604,True,2807.110315,1.524894e+09,0.503759,162,32340,33142.663530,291.061398,0.030505
9,email-eu3,3647777.0,12051.0,302.694963,True,725.054586,7.166190e+06,0.509026,89,8911,594.655215,9.035909,0.030083


In [64]:
from tnestmodel.temp_properties import burstiness

In [65]:
burstinesses = []
for dataset, G in zip(datasets, Gs):
    if dataset.name=="hospital_ward":
        G_out = G
    B = burstiness(G)
    burstinesses.append(B)
    print(dataset, B)

CSVDataset(aves-weaver-social, False) (0.8230845613465078, 0.8230845613465078, 0.8230845613465078)
CSVDataset(ht09, False) (0.7946953467708944, 0.7946953467708944, 0.7946953467708944)
CSVDataset(workplace_2013, False) (0.7325472002045527, 0.7325472002045527, 0.7325472002045527)
CSVDataset(mammalia-primate-association, False) (0.5707315120052295, 0.5707315120052295, 0.5707315120052295)
CSVDataset(mammalia-raccoon-proximity, False) (0.4584479062197504, 0.4584479062197504, 0.4584479062197504)
CSVDataset(talk_eo, True) (0.7456877685382545, 0.6274205783923539, 0.7264999853076153)
CSVDataset(dnc, True) (0.8623035309154276, 0.821485794869698, 0.8553770184336326)
CSVDataset(fb-forum, True) (0.6314172454500498, 0.536934306068052, 0.6128361343172726)
CSVDataset(email-eu2, True) (0.758320281839842, 0.7409402432692938, 0.811241321765743)
CSVDataset(email-eu3, True) (0.6658931689478838, 0.6461695676688121, 0.7353374598921935)


In [66]:
burstiness(G)

(0.6658931689478838, 0.6461695676688121, 0.7353374598921935)

In [67]:
# %%time
# calculator = GroupNodeByColor(G, h=10000)
# for d in range(0, 2):
  #  print("<<< depth", d)
   # calculator.prepare(d)
   # t, arr = G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True)
   # print(np.all(arr==89))

In [68]:
print(len(arr), (arr==88).sum())

8911 12


In [69]:
from tnestmodel.temp_rewire import rewire_temporal_graph_dir

In [70]:
def get_properties(G):
    B_in, B_out, B = burstiness(G)
    triangles = numba_get_all_triangles(G)
    c_triangles = causal_triangles(G)
    persistence = get_edge_persistence(G)
    num_times = len(G.times)
    num_edges = get_total_number_of_edges(G)
    num_nodes = G.num_nodes
    return B_in, B_out, B, triangles, c_triangles, persistence, num_times, num_edges, num_nodes

In [72]:
#def randomize_RTE_undir(G, n_steps):
#    for i in range(10):

In [93]:
def undir_stub_matching(G, seed=None, max_tries=1000):
    assert not G.is_directed
    if seed is not None:
        np.random.seed(seed)
    E = G.to_temporal_edges()
    counts1 = np.bincount(E[:,0].ravel(), minlength=G.num_nodes)
    counts2 = np.bincount(E[:,1].ravel(), minlength=G.num_nodes)
    num_edges = E.shape[0]
    tmp = np.empty(num_edges*2, dtype=E.dtype)
    tmp[0:num_edges] = E[:,0]
    tmp[num_edges:] = E[:,1]
    for num_tries in range(max_tries):
        tmp = tmp[np.random.permutation(2*num_edges)]
        out = tmp.reshape(num_edges, 2)
        if np.all(out[:,0]!=out[:,1]):
            print(num_tries)
            break
    return out
    
    

In [94]:
for dataset, G in zip(datasets, Gs):
    if not G.is_directed:
        break

In [99]:
%%time
undir_stub_matching(G, seed=1)

56
CPU times: total: 0 ns
Wall time: 4 ms


array([[230,  87],
       [344, 406],
       [399, 400],
       ...,
       [402, 396],
       [387, 274],
       [103, 273]], dtype=int64)

In [33]:
from numba import njit

In [35]:
from tnestmodel.temp_generation import randomize_keeping_aggregate_graph

In [36]:
from tnestmodel.temp_properties import _get_aggregated_graph

In [38]:
%%time
#%%snakeviz -t
num_seeds = 10
records_agg = []
for dataset, G in zip(datasets, Gs):
    agg_graph = _get_aggregated_graph(G, ignore_directionality=True) # need to ignore directionality, otherwise 
    for seed in range(num_seeds):
        for keep_multiplicities in [True, False]:
            print(dataset, seed, keep_multiplicities)
            E = randomize_keeping_aggregate_graph(agg_graph, G.times, keep_multiplicities, seed=seed)
            G_rew = SparseTempFastGraph.from_temporal_edges(E, G.is_directed)
            params = get_properties(G_rew)

            records_agg.append((dataset.abbr, seed, f"agg_{keep_multiplicities}", *params))

CSVDataset(aves-weaver-social, False) 0 True
CSVDataset(aves-weaver-social, False) 0 False
CSVDataset(aves-weaver-social, False) 1 True
CSVDataset(aves-weaver-social, False) 1 False
CSVDataset(aves-weaver-social, False) 2 True
CSVDataset(aves-weaver-social, False) 2 False
CSVDataset(aves-weaver-social, False) 3 True
CSVDataset(aves-weaver-social, False) 3 False
CSVDataset(aves-weaver-social, False) 4 True
CSVDataset(aves-weaver-social, False) 4 False
CSVDataset(aves-weaver-social, False) 5 True
CSVDataset(aves-weaver-social, False) 5 False
CSVDataset(aves-weaver-social, False) 6 True
CSVDataset(aves-weaver-social, False) 6 False
CSVDataset(aves-weaver-social, False) 7 True
CSVDataset(aves-weaver-social, False) 7 False
CSVDataset(aves-weaver-social, False) 8 True
CSVDataset(aves-weaver-social, False) 8 False
CSVDataset(aves-weaver-social, False) 9 True
CSVDataset(aves-weaver-social, False) 9 False
CSVDataset(ht09, False) 0 True
CSVDataset(ht09, False) 0 False
CSVDataset(ht09, False) 1 T

In [ ]:
df_agg = pd.DataFrame.from_records(records_agg, columns = ["abbr", "seed", "depth", "B_in", "B_out", "B", "triangles", "c_triangles", "persistence",  "num_times", "num_edges", "num_nodes"] )

In [ ]:
df_agg.to_csv("agg_graphs.csv")

In [ ]:
%%time
#%%snakeviz -t
num_seeds = 10
records = []
for dataset, G in zip(datasets, Gs):
    if G.is_directed:
        continue
    params = get_properties(G)
    records.append((dataset.abbr, -1, -1, *params))
    for seed in range(num_seeds):
        for depth in [0,1,len(G.s.colors_per_round)-1]:

            print(dataset, seed, depth)
            h = -1
            if not G.s is None:
                h = G.s.h
            G_rew = G.copy()
            G_rew.rewire_slices(depth=depth, seed=seed, method=1, r=10)
            
            params = get_properties(G_rew)

            records.append((dataset.abbr, seed, depth, *params))

In [ ]:
df_undir = pd.DataFrame.from_records(records, columns = ["abbr", "seed", "depth", "B_in", "B_out", "B", "triangles", "c_triangles", "persistence",  "num_times", "num_edges", "num_nodes"] )

In [ ]:
df_undir.to_csv("undirected_graphs.csv")

In [ ]:
%%time
#%%snakeviz -t
num_seeds = 10
records_2 = []
for dataset, G in zip(datasets, Gs):
    if not G.is_directed:
        continue
    params = get_properties(G)
    records_2.append((dataset.abbr, -1, -1, *params))
    for seed in range(num_seeds):
        for depth in [0,1,len(G.s.colors_per_round)-1]:

            print(dataset, seed, depth)
            h = -1
            if not G.s is None:
                h = G.s.h
            G_rew = rewire_temporal_graph_dir(G, d=depth, seed=seed, h=h)
            
            params = get_properties(G_rew)

            records_2.append((dataset.abbr, seed, depth, *params))

In [ ]:
df = pd.DataFrame.from_records(records_2, columns = ["abbr", "seed", "depth", "B_in", "B_out", "B", "triangles", "c_triangles", "persistence",  "num_times", "num_edges", "num_nodes"] )

In [ ]:
df

In [ ]:
df.to_csv("directed_graphs.csv")

In [ ]:
G_rew.to_temporal_edges()

In [ ]:
G.to_temporal_edges()

In [ ]:
G.num_nodes

In [ ]:
a,b,c = inter_arrival_times(G)

In [ ]:
plt.hist(np.log(b+1), bins=31)

In [ ]:
plt.hist(np.log(a+1), bins=30)

In [ ]:
G.to_temporal_edges()[:,2]

In [ ]:
print(datasets[5].mapping)

In [ ]:
#current_colors [313 247]
#set     colors [310 247]

In [ ]:
edges1 = [[2,1,2],
[3,4,2],
[3,1,1],
[4,3,2],
[0,2,3],
[1,2,2],
[1,0,3],
[2,0,3],
[1,3,1],
[0,1,3],]

In [ ]:
edges2=[[5,3,1],
[1,3,3],
[5,0,1],
[5,2,2],
[1,2,3],
[2,1,4],
[3,1,3],
[3,5,1],
[5,1,1],
[2,3,1],
[3,2,1],
[2,5,2],
[2,1,3],
[5,1,4],
[1,5,1],
[0,5,1],
[1,5,4],
[1,2,4],]

In [ ]:
G1 = SparseTempFastGraph.from_temporal_edges(np.array(edges2, dtype=int), is_directed=True)
m = get_aggregated_graph(G1)
c = count_triangles_in_aggregated(m)
print(c)

In [ ]:
fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize=(16,10))
fig.suptitle("number of edges")
axs = axs.ravel()
for ax, dataset, t, arr in zip(axs, datasets, ts, num_edges_per_slice):
    ax.scatter(t, arr)
    print(dataset, arr.sum())
    ax.set_title(dataset.name+" "+str(dataset.is_directed))
        #print(dataset.name)
        #print(arr[:,0].min())
plt.show()

In [ ]:
max_depth = max([len(x.slices[0].base_partitions) for x in Gs])


for d in range(max_depth):
    fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize=(16,10))
    fig.suptitle("number of possible dual flips")
    axs = axs.ravel()
    for ax, dataset, t, arr in zip(axs, datasets, ts, arrs):
        if arr.shape[1]>d:
            ax.plot(t, np.cumsum(arr[:,d]+1))
            ax.set_title(dataset.name)
            ax.set_yscale("log")
            #print(dataset.name)
            #print(arr[:,0].min())
    plt.show()

In [ ]:
np.max(arrs[0][:,0]-arrs[0][:,1])

In [ ]:
np.all(arrs[0][:,0]-arrs[0][:,1]>=0)

In [ ]:
np.max(arrs[0][:,1]-arrs[0][:,3])